In [62]:
# criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import pprint


nav = webdriver.Chrome()

In [59]:
# importar/ visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


Definição das funções de busca do google shopping e do buscapé

In [77]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False
    return tem_todos_termos_produtos

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_nome_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")
    lista_resultados = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    # entrar no google
    nav.get("https://www.google.com.br/")
    nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)
    # opção de busca por "class name"
    time.sleep(2)
    # entrar na aba shopping
    elementos = nav.find_elements("class name", "YmvwI")


    time.sleep(2)

    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    # pegar as informações do produto
    resultado_pesquisa = nav.find_elements('class name', 'i0X6df')

    for resultado in resultado_pesquisa:
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()
        
        # analizar se o produto tem algum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
                
        # analizar se o produto contem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)
        
        
        # filtro do valor
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = resultado.find_element('class name', 'a8Pemb').text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)
                        
            if preco_minimo<= preco <= preco_maximo:
                elemento_referencia = resultado.find_element('class name', 'bONr3b')
                elemento_pai = elemento_referencia.find_element('xpath', '..')
                link = elemento_pai.get_attribute('href')
                lista_resultados.append((nome, preco, link))
    return lista_resultados

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_nome_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")
    lista_resultado = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    nav.get("https://www.buscape.com.br/")
    nav.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    # pegar os resultados
    time.sleep(5)
    resultado_pesquisa = nav.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')
    
    for resultado in resultado_pesquisa:
        preco = resultado.find_element('class name', 'Text_MobileHeadingS__HEz7L').text
        nome = resultado.find_element('class name', 'ProductCard_ProductCard_Name__U_mUQ').text
        nome = nome.lower()
        link = resultado.get_attribute("href")
        
        # analizar se o produto tem algum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
       
        # analizar se o produto contem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)
        
        # analisar a lista de ofertas 
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)
            if preco_minimo <= preco <= preco_maximo:
                lista_resultado.append((nome, preco, link))
                
    return lista_resultado
        
    
       

Continuação do código

In [ ]:
# pesquizar pelo produto
produto = 'iphone 12 64 gb'
termos_banidos = "min watch"
preco_minimo = 2000
preco_maximo = 3500
# Lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
print("-" * 20)
pprint.pprint(lista_ofertas_buscape)
# pprint.pprint(Lista_ofertas_google_shopping)

# para cada item dentro da nossa base de dados, para cada produto

    # procurar esse produto no google shooping
        # verificar se algum dos produtos do google shooping está dentro da minha faixa de preço
    
    #procurar esse produto no buscapé
        # verificar se algum dos produtos do buscapé está dentro da minha faixa de preço
        
# salvar as ofertas boas em um dataframe (tabela)

# exportar pro excel

# enviar por email o resultado da tabela